In [21]:
import os
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from plyfile import PlyData, PlyElement
import tqdm
from meshplot import plot
import sys 
sys.path.append('../')
import radfoam
import mesh_tools as mt
from radfoam_model.mesh_utils import marching_tetrahedra

device='cuda'

In [22]:
v, f, samples = mt.load_and_sample_shape('pig.obj', '../data/',)
queries = torch.tensor(samples, device='cuda', dtype=torch.float32) 

plot(v,f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0656904…

In [23]:
class AugmentedTriangulation:
    def __init__(self, primal_points: torch.tensor) -> None:
        self.triangulation = radfoam.Triangulation(primal_points)
        perm = self.triangulation.permutation().to(torch.long)
        self.points = primal_points[perm] 
        self.tree = radfoam.build_aabb_tree(self.points)
        
    @property
    def tets(self):
        return self.triangulation.tets().long()
    
    @property
    def tets_neighbors(self):
        neighs = ((self.triangulation.tet_adjacency().long())//4)
        neighs[neighs>len(self.tets)] = -1
        return neighs
    
    @property
    def triangle_faces(self):
        opp_face = torch.tensor([[1, 2, 3], [0, 3, 2], [0, 1, 3], [0, 2, 1]], device=device)
        return self.tets[:, opp_face]
    
    @property 
    def tet_bary(self):
        return self.points[self.tets].mean(-2)
    
    def signed_volume(self, P, queries):
        '''TODO DEBUG'''
        p1 = P[:,:,1] - P[:,:,0]
        p2 = P[:,:,2] - P[:,:,0]
        p3 = queries[:, None] - P[:,:,0]
        return torch.det(torch.stack((p1, p2, p3), dim=-1))

    def signed_normal(self, P, queries):
        p1 = P[:,:,1] - P[:,:,0]
        p2 = P[:,:,2] - P[:,:,0]
        face_normal = torch.cross(p1, p2, dim=-1)
        sign = ((queries[:, None]-P.sum(-2))*face_normal).sum(-1)
        return sign
        
    def query_to_tet(self, queries, max_iter=100):
        nearest_neighbor = radfoam.nn(self.points, self.tree, queries)
        tet_index = self.triangulation.vert_to_tet().long()[nearest_neighbor.long()]
        to_see = torch.ones(len(queries), dtype=torch.bool, device=queries.device)
        for i in range(max_iter):
            P = self.points[self.triangle_faces[tet_index[to_see]]] 
            signed_volume = self.signed_normal(P, queries[to_see])
            smin, sargmin = signed_volume.min(-1)
            to_see[torch.arange(len(to_see), device=device)[to_see][smin>=0]] = False
            tet_index[to_see] = self.tets_neighbors[tet_index[to_see], sargmin[smin<0]]
            if to_see.sum()==0:
                break
        return tet_index

num_init_points = 30000

primal_points = (torch.rand(num_init_points, 3, device=device)-.5)*2.

triangulation = radfoam.Triangulation(primal_points)
perm = triangulation.permutation().long()
primal_points = primal_points[perm]

augmented_triangulation = AugmentedTriangulation(primal_points)
query_tet = augmented_triangulation.query_to_tet(queries)
# augmented_triangulation.get_triangle_faces()


In [25]:
primal_values = torch.ones_like(primal_points[:, 0])
primal_values[augmented_triangulation.tets[query_tet].unique()] = -1
nv, nf, _, edge = marching_tetrahedra(triangulation.tets().long(),primal_values,primal_points, primal_points, True)

plot(nv.cpu().detach().numpy(), nf.cpu().detach().numpy())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0194812…